# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os

# Import API key
geoapify_key = os.getenv('geoapify_api_key')

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ilulissat,69.2167,-51.1000,6.01,56,7,0.51,GL,1727055359
1,1,bethel,41.3712,-73.4140,16.42,75,0,4.12,US,1727055337
2,2,waitangi,-43.9535,-176.5597,11.35,86,76,6.32,NZ,1727055361
3,3,barczewo,53.8306,20.6911,12.37,67,7,2.95,PL,1727055362
4,4,adamstown,-25.0660,-130.1015,20.75,78,7,2.72,PN,1727055364


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
#%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 'Humidity',
    color = 'City',
    alpha = 0.6,
    frame_width = 800,
    frame_height = 600    
)


# Display the map
city_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 15
max_temp = 28
min_humidity = 40
max_humidity = 60
max_wind_speed = 2
max_cloudiness = 10

# Drop any rows with null values
filtered_cities_df = city_data_df.dropna(subset=['Max Temp', 'Humidity','Wind Speed','Cloudiness'])

filtered_cities_df = filtered_cities_df[
    (filtered_cities_df['Max Temp'] >= min_temp) &
    (filtered_cities_df['Max Temp'] <= max_temp) &
    (filtered_cities_df['Humidity'] >= min_humidity) &
    (filtered_cities_df['Humidity'] <= max_humidity)&
    (filtered_cities_df['Cloudiness'] <= max_cloudiness)
].copy()
# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,hede,33.7681,120.2517,22.65,58,0,4.63,CN,1727055390
40,40,suncho corral,-27.9336,-63.4294,25.97,46,0,9.94,AR,1727055405
51,51,scottsbluff,41.8666,-103.6672,17.13,50,0,4.63,US,1727055417
57,57,ballina,-28.8667,153.5667,22.12,55,0,4.09,AU,1727055423
68,68,saldanha,-33.0117,17.9442,16.03,55,7,3.87,ZA,1727055436
119,119,urangan,-25.3000,152.9000,23.32,58,0,3.60,AU,1727055493
126,126,constantia,44.1833,28.6500,16.50,57,6,4.18,RO,1727055501
131,131,macheng,31.1783,115.0319,23.12,47,2,2.34,CN,1727055507
135,135,shizuishan,39.2333,106.7694,17.76,44,0,2.27,CN,1727055511
251,251,saint-pierre,-21.3393,55.4781,20.82,56,0,3.09,RE,1727055646


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
28,hede,CN,33.7681,120.2517,58,
40,suncho corral,AR,-27.9336,-63.4294,46,
51,scottsbluff,US,41.8666,-103.6672,50,
57,ballina,AU,-28.8667,153.5667,55,
68,saldanha,ZA,-33.0117,17.9442,55,
119,urangan,AU,-25.3000,152.9000,58,
126,constantia,RO,44.1833,28.6500,57,
131,macheng,CN,31.1783,115.0319,47,
135,shizuishan,CN,39.2333,106.7694,44,
251,saint-pierre,RE,-21.3393,55.4781,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key,
    'categories': 'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hede - nearest hotel: No hotel found
suncho corral - nearest hotel: No hotel found
scottsbluff - nearest hotel: Hampton Inn & Suites Scottsbluff Conference Center
ballina - nearest hotel: Ballina Manor Boutique Hotel
saldanha - nearest hotel: Hoedjesbaai
urangan - nearest hotel: Ramada
constantia - nearest hotel: Hotel Dali
macheng - nearest hotel: No hotel found
shizuishan - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
urzhar - nearest hotel: Гостиница Шанырак
yatou - nearest hotel: 盛悦商务酒店
al jawf - nearest hotel: No hotel found
birobidzhan - nearest hotel: Мини-гостиница
great falls - nearest hotel: La Quinta Inn & Suites
chinsali - nearest hotel: No hotel found
minot - nearest hotel: The Dakota Inn Minot, ND


,City,Country,Lat,Lng,Humidity,Hotel Name
28,hede,CN,33.7681,120.2517,58,No hotel found
40,suncho corral,AR,-27.9336,-63.4294,46,No hotel found
51,scottsbluff,US,41.8666,-103.6672,50,Hampton Inn & Suites Scottsbluff Conference Ce...
57,ballina,AU,-28.8667,153.5667,55,Ballina Manor Boutique Hotel
68,saldanha,ZA,-33.0117,17.9442,55,Hoedjesbaai
119,urangan,AU,-25.3000,152.9000,58,Ramada
126,constantia,RO,44.1833,28.6500,57,Hotel Dali
131,macheng,CN,31.1783,115.0319,47,No hotel found
135,shizuishan,CN,39.2333,106.7694,44,No hotel found
251,saint-pierre,RE,-21.3393,55.4781,56,Tropic Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
#%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 'Humidity',
    color = 'City',
    alpha = 0.6,
    hover_cols =['Hotel Name', 'Country'],
    frame_width = 800,
    frame_height = 600    
)


# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)